# EMG WORD PREDICTION

### IMPORT NECESSARY MODULES

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
import os
import glob
import pickle
import tqdm
import warnings
warnings.filterwarnings("ignore")

%matplotlib inline
import matplotlib as mlp
import matplotlib.pyplot as plt
mlp.rc("xtick",labelsize=12)
mlp.rc("ytick",labelsize=12)
mlp.rc("axes",labelsize=14)

plt.rcParams['figure.figsize'] = [10,5]
plt.rcParams['figure.dpi'] = 300

from scipy import signal
from scipy.stats import zscore
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

import librosa

from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

from sklearn.decomposition import FastICA
from sklearn.decomposition import PCA

from sklearn.svm import SVC

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import roc_auc_score

from sklearn.model_selection import StratifiedKFold
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV


from sklearn.pipeline import Pipeline

import tensorflow as tf
from tensorflow import keras

CURR_DIR = os.getcwd()

In [2]:

MAIN_DIR = os.path.join(CURR_DIR,"..","..")
DATA_DIR = os.path.join(MAIN_DIR,"dataset")
FILE_DIR = os.path.join(MAIN_DIR,"dataset","pickle")
FIG_DIR = os.path.join(MAIN_DIR,"figures")
os.makedirs(FIG_DIR,exist_ok=True)

SPEAKER = ["RL","RN","SR","US"]
MODE = ["mentally","mouthed"]
WORDS = LABELS = ["add","call","go","later","left","reply","right","stop","subtract","you"]

SAMPLING_FREQ = 250 
NUM_CHANNELS = 8


# a function to save plotted figures
def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(FIG_DIR, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)


In [10]:
DATASET = os.listdir(FILE_DIR)
DATASET.sort()
print(DATASET[-6:])

['US_feature_spectral_me.pickle', 'US_feature_spectral_mo.pickle', 'US_feature_time_me.pickle', 'US_feature_time_mo.pickle', 'US_feature_time_spectral_me.pickle', 'US_feature_time_spectral_mo.pickle']


In [14]:

# feature_spectral_mental = pickle.load(open(os.path.join(FILE_DIR,DATASET[-6]),"rb"))
# feature_spectral_mouthed = pickle.load(open(os.path.join(FILE_DIR,DATASET[-3]),"rb"))
# feature_time_mental = pickle.load(open(os.path.join(FILE_DIR,DATASET[-4]),"rb"))
feature_time_mouthed = pickle.load(open(os.path.join(FILE_DIR,DATASET[-3]),"rb"))
# feature_time_spectral_mental = pickle.load(open(os.path.join(FILE_DIR,DATASET[4]),"rb"))
# feature_time_spectral_mental = pickle.load(open(os.path.join(FILE_DIR,DATASET[5]),"rb"))

In [15]:
# feature_time_mental.keys()
feature_time_mouthed.keys()

dict_keys(['data', 'label'])

In [19]:
X_spectral = feature_spectral_mouthed["data"]
Y_spectral = feature_spectral_mouthed["label"]

X_time = feature_time_mouthed["data"]
Y_time = feature_time_mouthed["label"]

# X_spectral = feature_spectral_mental["data"]
# Y_spectral = feature_spectral_mental["label"]

# X_time = feature_time_mental["data"]
# Y_time = feature_time_mental["label"]

In [20]:

print(X_spectral.shape)
print(Y_spectral.shape)

print(X_time.shape)
print(Y_time.shape)

(1765, 920, 40)
(1765,)
(1765, 920, 40)
(1765,)


### LABEL ENCODING (MLP AND CNN)

In [21]:
encoder = LabelEncoder()
Y = encoder.fit_transform(Y_spectral)
# Y_1hot = tf.keras.utils.to_categorical(Y_spectral, num_classes = 10)
print(encoder.classes_)
print("Encoded labels (normal): \n",Y[0:5])
# print("Encoded labels (ONE-HOT): \n",Y_1hot[0:5])

# encoder = LabelEncoder()
# Y_time = encoder.fit_transform(Y_time)
# Y_1hot_time = tf.keras.utils.to_categorical(Y_time, num_classes = 10)
# print(encoder.classes_)
# print("Encoded labels (normal): \n",Y_time[0:5])
# print("Encoded labels (ONE-HOT): \n",Y_1hot_time[0:5])

# encoder = LabelEncoder()
# Y_spectral = encoder.fit_transform(Y_spectral)
# Y_1hot_spectral = tf.keras.utils.to_categorical(Y_spectral, num_classes = 10)
# print(encoder.classes_)
# print("Encoded labels (normal): \n",Y_spectral[0:5])
# print("Encoded labels (ONE-HOT): \n",Y_1hot_spectral[0:5])

['add' 'call' 'go' 'later' 'left' 'reply' 'right' 'stop' 'subtract' 'you']
Encoded labels (normal): 
 [1 1 1 1 1]


In [ ]:
def train_test_split(X,Y):
    split = StratifiedShuffleSplit(n_splits=1, test_size =0.1, random_state=42)
    train_id, test_id = next(split.split(X,Y))
    return np.array([X[n] for n in train_id]), np.array([Y[n] for n in train_id]), np.array([X[n] for n in test_id]),np.array([Y[n] for n in test_id])

In [ ]:
def plotCM(y, predictions,y2,predictions2,filename=" "):
    cm = confusion_matrix(y, predictions)
#     print(cm)
#     plt.suptitle("KNN\n\n")
    plt.subplot(121)
    plt.title("With Temporal Features")
    plt.imshow(confusion_matrix(y, predictions,normalize="true"), interpolation = 'nearest',cmap = plt.cm.Reds)
    plt.xticks(range(len(LABELS)), LABELS, rotation = 45)
    plt.yticks(range(len(LABELS)), LABELS)
    
    plt.ylabel('Actual Label')
    plt.xlabel('Predicted Label')

    for i in range(10):
        for j in range(10):
            plt.text(j,i, cm[i][j],horizontalalignment='center',verticalalignment='center')
   

    plt.subplot(122)
    plt.title("With Spectral Features")
    plt.imshow(confusion_matrix(y2, predictions2,normalize="true"), interpolation = 'nearest',cmap = plt.cm.Reds)
    plt.xticks(range(len(LABELS)), LABELS, rotation = 45)
    plt.yticks(range(len(LABELS)), LABELS)
    
#     plt.ylabel('Actual Label')
    plt.xlabel('Predicted Label')
    cm = confusion_matrix(y2, predictions2)
    
    for i in range(10):
        for j in range(10):
            plt.text(j,i, cm[i][j],horizontalalignment='center',verticalalignment='center')
    
    if(filename!=" "):
        save_fig(filename)
    plt.subplots_adjust(wspace=0.3)
    plt.show()

In [53]:
def K_fold_CV(X_new,Y_new,K=10):
    kfold = StratifiedKFold(n_splits=K,random_state=42)
    acc = []
    for train_id,test_id in kfold.split(X_new,Y_new):
        #acc.append(KNN_Classifier(X_new[train_id],Y_new[train_id],X_new[test_id],Y_new[test_id]))
        acc.append(MLP_Classifier(X_new[train_id],Y_new[train_id],X_new[test_id],Y_new[test_id]))
        #acc.append(CNN_Classifier(X_new[train_id],Y_new[train_id],X_new[test_id],Y_new[test_id]))
    return np.array(acc)
    

In [44]:
# K fold




## MLP MODEL DEVELOPMENT

### FLATTEN DATA FOR MLP

In [45]:

X_MLP_time = np.array([x.flatten() for x in X_time])
print("Raw feature shape: ",X_MLP_time.shape)


X_MLP_spectral = np.array([x.flatten() for x in X_spectral])
print("Raw feature shape: ",X_MLP_spectral.shape)

Raw feature shape:  (1765, 36800)
Raw feature shape:  (1765, 36800)


### SCALE DATA

In [46]:
scalar = StandardScaler()
X_MLP_time = scalar.fit_transform(X_MLP_time,Y)
X_MLP_spectral = scalar.fit_transform(X_MLP_spectral,Y)

In [47]:
reducer = FastICA(n_components=100,random_state=42)
X_MLP_time = reducer.fit_transform(X_MLP_time)
X_MLP_spectral = reducer.fit_transform(X_MLP_spectral)

In [48]:
# X_train_time,Y_train_time,X_test_time,Y_test_time = train_test_split(X_MLP_time,Y)

# print("Train Set Shape of X: ",X_train_time.shape)
# print("Test Set Shape of X: ",X_test_time.shape)

# print("Train Set Shape of Y: ",Y_train_time.shape)
# print("Test Set Shape of Y: ",Y_test_time.shape)


# X_train_spectral,Y_train_spectral,X_test_spectral,Y_test_spectral = train_test_split(X_MLP_spectral,Y)

# print("Train Set Shape of X: ",X_train_spectral.shape)
# print("Test Set Shape of X: ",X_test_spectral.shape)

# print("Train Set Shape of Y: ",Y_train_spectral.shape)
# print("Test Set Shape of Y: ",Y_test_spectral.shape)

### DEFINE MODEL

In [51]:
def MLP_Classifier(X_train,Y_train,X_test,Y_test,PLOTS=True):
    MLP_model = keras.Sequential()
    MLP_model.add(keras.layers.Dense(units=200,activation="relu",input_shape=X_train[0].shape))
    MLP_model.add(keras.layers.Dropout(0.5))
    MLP_model.add(keras.layers.Dense(64,activation="relu"))
    MLP_model.add(keras.layers.Dense(64,activation="relu"))
    MLP_model.add(keras.layers.Dropout(0.5))
    MLP_model.add(keras.layers.Dense(10,activation="softmax"))
    print(MLP_model.summary())
    optimizer=keras.optimizers.Adam()
    MLP_model.compile(optimizer=optimizer,loss="sparse_categorical_crossentropy",metrics=["accuracy"])
    history = MLP_model.fit(X_train,Y_train,epochs=30,validation_data=(X_test,Y_test),verbose=1)    
    MLP_prediction = np.array(list(map(np.argmax,MLP_model.predict(X_test))))
    
#     if(PLOTS==True):
#         plt.plot(history.history["accuracy"],"r:")
#         plt.plot(history.history["loss"],"b--")
#         plt.xlabel("Epoch")
#         plt.ylabel("Accuracy/ Loss")
#         plt.legend(["Accuracy","Loss"])
    # save_fig("ACC_LOSS_US_FIltered")

    return Y_test,MLP_prediction,history.history['accuracy']

In [55]:
# #K_fold_CV(X_new,Y_new,K=10)

# a = K_fold_CV(X_time,Y_time)
# a


(177,)

In [ ]:
a,b,MLP_history_time = MLP_Classifier(X_train_time,Y_train_time,X_test_time,Y_test_time)
c,d,MLP_history_spectral = MLP_Classifier(X_train_spectral,Y_train_spectral,X_test_spectral,Y_test_spectral)

In [ ]:
# len(MLP_history_spectral)

In [ ]:
plotCM(a,b.flatten(),c,d.flatten(),"US_mental_MLP")


In [ ]:

plt.subplot(121)
# plt.title("With Temporal Features")
plt.plot(MLP_history_time.history["loss"],"b:")
plt.plot(MLP_history_spectral.history["loss"],"r--")
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(["Temporal","Spectral"])

plt.subplot(122)
# plt.title("With Temporal Features")
plt.plot(MLP_history_time.history["accuracy"],"b:")
plt.plot(MLP_history_spectral.history["accuracy"],"r--")
plt.ylabel('Accuracy')
plt.xlabel('Epoch')

plt.subplots_adjust(wspace=0.3)
plt.legend(["Temporal","Spectral"])
save_fig("US_mental_MLP_ACCURACY_LOSS")
plt.show()

In [ ]:
print("Precision/Recall with Temporal Features")
print(classification_report(a,b))

print("Precision/Recall with Spectral Features")
print(classification_report(c,d))

## CNN MODEL DEVELOPMENT

In [ ]:
from copy import deepcopy

X_CNN_spectral = deepcopy(X_spectral)

X_CNN_time = deepcopy(X_time)

print(X_CNN_time.shape)

print(X_CNN_spectral.shape)

### SCALING DATA

In [ ]:
scalar = StandardScaler()
X_CNN_time = (scalar.fit_transform(X_CNN_time.reshape(X_CNN_time.shape[0],-1),Y)).reshape(X_CNN_time.shape[0],X_CNN_time.shape[1],-1)
X_CNN_spectral = (scalar.fit_transform(X_CNN_spectral.reshape(X_CNN_time.shape[0],-1),Y)).reshape(X_CNN_time.shape[0],X_CNN_time.shape[1],-1)

In [ ]:

print(X_CNN_time.shape)

print(X_CNN_spectral.shape)

In [ ]:
X_train_time,Y_train_time,X_test_time,Y_test_time = train_test_split(X_CNN_time,Y)
print("Train Set Shape: ",X_train_time.shape)
print("Test Set Shape: ",X_test_time.shape)

print("Train Set Shape: ",Y_train_time.shape)
print("Test Set Shape: ",Y_test_time.shape)


X_train_spectral,Y_train_spectral,X_test_spectral,Y_test_spectral = train_test_split(X_CNN_spectral,Y)
print("Train Set Shape: ",X_train_spectral.shape)
print("Test Set Shape: ",X_test_spectral.shape)

print("Train Set Shape: ",Y_train_spectral.shape)
print("Test Set Shape: ",Y_test_spectral.shape)

In [ ]:
Y_train_spectral = (Y_train_spectral.reshape(-1,len(Y_train_spectral))).T
Y_test_spectral = (Y_test_spectral.reshape(-1,len(Y_test_spectral))).T

print("Train Set Shape: ",X_train_spectral.shape)
print("Test Set Shape: ",X_test_spectral.shape)

print("Train Set Shape: ",Y_train_spectral.shape)
print("Test Set Shape: ",Y_test_spectral.shape)


Y_train_time = (Y_train_time.reshape(-1,len(Y_train_time))).T
Y_test_time = (Y_test_time.reshape(-1,len(Y_test_time))).T

# print("Train Set Shape: ",X_train_spectral.shape)
# print("Test Set Shape: ",X_test_spectral.shape)

# print("Train Set Shape: ",Y_train_spectral.shape)
# print("Test Set Shape: ",Y_test_spectral.shape)

In [ ]:
def CNN_Classifier(X_train,Y_train,X_test,Y_test,PLOTS=False):
    n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], Y_train.shape[1]
    CNN_model = keras.Sequential()
    CNN_model.add(keras.layers.Conv1D(100, kernel_size = 12, input_shape =(n_timesteps,n_features), activation = "relu"))
    # 	CNN_model.add(keras.layers.Conv1D(100, kernel_size = 12, input_shape = X_train.shape[1:], activation = "relu"))
    CNN_model.add(keras.layers.MaxPool1D(pool_size=2))
    CNN_model.add(keras.layers.Conv1D(100,kernel_size=6,activation="relu"))
    CNN_model.add(keras.layers.MaxPool1D(pool_size=2))
    CNN_model.add(keras.layers.Flatten())
    CNN_model.add(keras.layers.Dense(100,activation="relu"))
    CNN_model.add(keras.layers.Dense(10,activation="softmax"))

    opt = keras.optimizers.Adam()

    CNN_model.compile(optimizer = opt, loss = keras.losses.sparse_categorical_crossentropy, metrics=['accuracy'])
    print(CNN_model.summary())
    history = CNN_model.fit(X_train, Y_train, epochs = 10, batch_size = 50, validation_data =(X_test, Y_test) ,verbose = 1)
    CNN_prediction = np.array(list(map(np.argmax,CNN_model.predict(X_test))))
    return Y_test,CNN_prediction,history

In [ ]:
e,f,history_time = CNN_Classifier(X_train_time,Y_train_time,X_test_time,Y_test_time)
g,h, history_spectral = CNN_Classifier(X_train_spectral,Y_train_spectral,X_test_spectral,Y_test_spectral)

In [ ]:
plotCM(e,f.flatten(),g,h.flatten(),"US_CNN_mental_10_epoch")
# print(b.flatten())

In [ ]:

plt.subplot(121)
# plt.title("With Temporal Features")
plt.plot(history_time.history["loss"],"b:")
plt.plot(history_spectral.history["loss"],"r--")
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(["Temporal","Spectral"])

plt.subplot(122)
# plt.title("With Temporal Features")
plt.plot(history_time.history["accuracy"],"b:")
plt.plot(history_spectral.history["accuracy"],"r--")
plt.ylabel('Accuracy')
plt.xlabel('Epoch')

plt.subplots_adjust(wspace=0.3)
plt.legend(["Temporal","Spectral"])
save_fig("US_mental_CNN_ACCURACY_LOSS_10_epoch")
plt.show()

In [ ]:
print("Precision/Recall with Temporal Features")
print(classification_report(e,f))

print("Precision/Recall with Spectral Features")
print(classification_report(g,h))

In [ ]:
# type(X_time)

# #reshaping for 2d cnn
# XX_time = np.zeros((X_time.shape[0], X_time.shape[1], int(X_time.shape[-1]/8), 8))
# YY_time = Y_time



# for i in range(len(XX_time)):
#     for chan in range(XX_time.shape[-1]):
#         for feat in range(XX_time.shape[-2]):
#             XX_time[i, :, feat, chan] = X_time[i, :, XX_time.shape[-2]*chan + feat ]

In [ ]:
# def CNN_Classifier_2D(X_train,Y_train,X_test,Y_test,PLOTS=False):
#     Y_train = tf.keras.utils.to_categorical(Y_train, num_classes = 10)
#     Y_test = tf.keras.utils.to_categorical(Y_test, num_classes = 10)
    
# #     n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], Y_train.shape[1]
#     CNN_model = keras.Sequential()
# #     CNN_model.add(keras.layers.Conv2D(100, kernel_size = 12, input_shape =(n_timesteps,n_features), activation = "relu"))
#     CNN_model.add(keras.layers.Conv2D(100, kernel_size = (5,5), input_shape = X_train.shape[1:], activation = "relu"))
#     CNN_model.add(keras.layers.MaxPool2D(pool_size=(1, 1)))
#     CNN_model.add(keras.layers.Conv2D(100,kernel_size= (6,1),activation="relu"))
#     CNN_model.add(keras.layers.MaxPool2D(pool_size= (1, 1) ))
#     CNN_model.add(keras.layers.Flatten())
#     CNN_model.add(keras.layers.Dense(100,activation="relu"))
#     CNN_model.add(keras.layers.Dense(10,activation="softmax"))

#     opt = keras.optimizers.Adam()

#     CNN_model.compile(optimizer = opt, loss = keras.losses.sparse_categorical_crossentropy, metrics=['accuracy'])
#     print(CNN_model.summary())
#     history = CNN_model.fit(X_train, Y_train, epochs = 30, batch_size = 50, validation_data =(X_test, Y_test) ,verbose = 1)
#     CNN_prediction = np.array(list(map(np.argmax,CNN_model.predict(X_test))))
    
# #     if(PLOTS=True):
# #         plt.plot(history.history["accuracy"],"r:")
# #         plt.plot(history.history["loss"],"b--")
# #         plt.xlabel("Epoch")
# #         plt.ylabel("Accuracy/ Loss")
# #         plt.legend(["Accuracy","Loss"])
# #         # save_fig("ACC_LOSS_US_FIltered")
    
    
#     return Y_test,CNN_prediction,history

In [ ]:
# X_train_time,Y_train_time,X_test_time,Y_test_time = train_test_split(XX_time,Y_en)
# a,b,history_time = CNN_Classifier_2D(X_train_time,Y_train_time,X_test_time,Y_test_time)